In [ ]:
Overview
You are building an application that consists of many different services that can depend on each other. One of these services is the entrypoint which receives user requests and then makes requests to each of its dependencies, which will in turn call each of their dependencies and so on before returning.
Given a directed acyclic graph that contains these dependencies, you are tasked with determining the "load factor" for each of these services to handle this load. The load factor of a service is defined as the number of units of load it will receive if the entrypoint receives a 1 unit of load. Note that we are interested in the worst case capacity. For a given downstream service, its load factor is the number of units of load it is required to handle if all upstream services made simultaneous requests. For example, in the following dependency graph where A is the entrypoint:

Each query to A will generate one query to B which will pass it on to C and from there to D. A will also generate a query to C which will pass it on to D, so the worst case (maximum) load factors for each service is A:1, B:1, C:2, D:2.
(Important: make sure you've fully understood the above example before proceeding!)

Problem Details

service_list: An array of strings of format service_name=dependency1,dependency2. Dependencies can be blank (e.g. dashboard=) and non-existent dependency references should be ignored (e.g. prices=users,foobar and foobar is not a service defined in the graph). Each service is defined only once in the graph.
entrypoint: An arbitrary service that is guaranteed to exist within the graph
Output: A list of all services depended by (and including) entrypoint as an array of strings with the format service_name*load_factor sorted by service name.
Example

Input:
service_list = ["logging=",
"user=logging",
"orders=user,foobar",
"recommendations=user,orders",
"dashboard=user,orders,recommendations"]
entrypoint = "dashboard"

Output (note sorted by service name)
["dashboard1",
"logging4",
"orders2",
"recommendations1",
"user*4"]
[execution time limit] 3 seconds (cs)

[input] array.string service_list

[input] string entrypoint

[output] array.string

[C#] Syntax Tips

// Prints help message to the console
// Returns a string
string helloWorld(string name) {
Console.Write("This prints to the console when you Run Tests");
return "Hello, " + name;
}

In [17]:
from collections import defaultdict
def load_factor(service_list, entrypoint):
    existed_dependencies = set(entrypoint)
    dependencies = {}
    service_loads = defaultdict(int)
    for service in service_list:
        parent, children = service.split("=")
        existed_dependencies.add(parent)
        # todo handle duplicates
        if children:
            dependencies[parent] = children.split(",")
        else:
            dependencies[parent] = []


    def dfs(service, visited):
        if service not in existed_dependencies:
            return
        if visited[service]: # if it returns True it's a cycle
            return
        service_loads[service] += 1
        visited[service] = True
        for child in dependencies[service]:
            dfs(child, visited)
        visited[service] = False

    
    dfs(entrypoint, defaultdict(lambda: False))
    res = [service+""+str(load) for service,load in sorted(service_loads.items(), key=lambda item: item[0])]
    return res



In [18]:
service_list_0 = ["logging=",
"user=logging,orders",
"orders=user,foobar",
"recommendations=user,orders",
"dashboard=user,orders,recommendations"]
entrypoint_0 = "dashboard"
load_factor(service_list_0, entrypoint_0)

['dashboard1', 'logging4', 'orders4', 'recommendations1', 'user4']

In [ ]:
# my interview answer
class Service:
    def __init__(self, name):
        self.name = name
        self.children = []
        self.load_factor = 0



def solution(service_list, entrypoint):
    existed_references = set()
    services = {} # mapping from name to service object
    def construct_graph():
        for service_child in service_list:
            parent, children_list = service_child.split("=")
            existed_references.add(parent)
            if parent not in services:
                services[parent] = Service(parent)
            if children_list != "":
                for child in children_list.split(","):
                    if child not in services:
                        services[child] = Service(child)
                    services[parent].children.append(services[child])
                        
        
    construct_graph()
    
    entry_point_dependencies = set()
    def dfs(service):
        entry_point_dependencies.add(service)
        for child in service.children:
            if child.name not in existed_references:
                return
            child.load_factor += 1
            dfs(child)
        return
        
    dfs(services[entrypoint])
    services[entrypoint].load_factor = 1
    
    load_factors = [service.name + "*" + str(service.load_factor) for service in entry_point_dependencies]
    load_factors.sort()
    return load_factors
    
                